In [1]:
import os
import sys
sys.path.append("..")
from src.embed import STextEmbedder
import pandas as pd
import weaviate
import weaviate.classes.config as wc
import torch

In [2]:
# create a collection locally with 
client = weaviate.connect_to_local()
embedder = STextEmbedder(embedder_name="all-mpnet-base-v2", device="cuda")

In [5]:
client.collections.delete("hnsl_vec_db")

In [3]:
client.collections.create(
    name="hnsl_vec_db",
    properties=[
        wc.Property(name="title", data_type=wc.DataType.TEXT),
        wc.Property(name="url", data_type=wc.DataType.TEXT),
        wc.Property(name="chunk_uuid", data_type=wc.DataType.UUID)
    ]
)

UnexpectedStatusCodeError: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'class name Hnsl_vec_db already exists'}]}.

In [6]:
client.collections.exists("hnsl_vec_db")

True

In [4]:
collection = client.collections.get("hnsl_vec_db")

In [5]:
df = pd.read_csv("../data/yamrzou_links.csv")

In [6]:
from uuid import uuid4
for index, row in df.iterrows():
    filename = row['title'].lower().replace(" ", "_")
    filename = filename.replace(".", "_")
    try:
        with open(f"../data/contents/{filename}.txt", "r") as f:
            corpus = f.read()
        embeddings = embedder.embed(corpus)
        for embedding in embeddings:
            chunk_uuid = uuid4()
            collection.data.insert(
            properties={
                "Title": row['title'],
                "url": row['url'],
                "chunk_uuid": chunk_uuid
            },
            vector=embedding.tolist(),
        )
        if index % 100 == 0:
            print(f"Finished indexing URL {index} / {len(df)}")    
    except Exception as e:
        pass

Finished indexing URL 100 / 2773
Finished indexing URL 200 / 2773
Finished indexing URL 300 / 2773


/home/farouk/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/home/farouk/anaconda3/lib/python3.12/ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


KeyboardInterrupt: 

In [12]:
query_embedding.shape

(384,)

In [12]:
import weaviate.classes as wvc

query = "How not to become rich?"
query_embedding = embedder.embed_query(query)
results = collection.query.near_vector(
            near_vector=query_embedding.tolist(),
            limit=10,
            return_metadata=wvc.query.MetadataQuery(certainty=True),
            return_properties=['title', 'url'],
        )
for result in results.objects:
    print(f"{result.properties['title']} - {result.properties['url']}")

How to Make Wealth (2004) - http://www.paulgraham.com/wealth.html
How to Make Wealth (2004) - http://www.paulgraham.com/wealth.html
Enjoy the Small Things in Life (Avoid the Connoisseur Effect) - https://www.tictoclife.com/enjoy-the-small-things/
Alan Watts on Money vs. Wealth (2014) - https://www.themarginalian.org/2014/05/29/alan-watts-on-money-vs-wealth/
How to Make Wealth (2004) - http://www.paulgraham.com/wealth.html
How to Make Wealth (2004) - http://www.paulgraham.com/wealth.html
Enjoy the Small Things in Life (Avoid the Connoisseur Effect) - https://www.tictoclife.com/enjoy-the-small-things/
How to Make Wealth (2004) - http://www.paulgraham.com/wealth.html
Moore's Law for Everything (2021) - https://moores.samaltman.com/
How to Make Wealth (2004) - http://www.paulgraham.com/wealth.html
